In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column+1):
    if ws.cell(1, column).value == 'Src':
        src_index = column
    elif ws.cell(1, column).value == 'Pic':
        pic_index = column
        
print('Src索引：' + str(src_index))
print('Pic索引：' + str(pic_index))
print()

# = = = = = = = = = = = = = = =

import requests
from io import BytesIO
from openpyxl.drawing.image import Image

# = = = = = = = = = = = = = = =

from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

work = Queue()
for row in range(2, ws.max_row+1):
    work.put_nowait(row)

def crawler():
    global ws
    
    while not work.empty():
        row = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        src = ws.cell(row, src_index).value
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            if src == None or src == '':
                break
            
            try:
                pic = Image(BytesIO(requests.get(src, proxies=Proxy.get_Proxy_Requests()).content))
                
                # = = = = = = = = = = = = = = = = = =
                
                if pic.width >= pic.height:
                    pic.height = 50.0 / pic.width * pic.height
                    pic.width = 50.0
                else:
                    pic.width = 50.0 / pic.height * pic.width
                    pic.height = 50.0
                    
                # = = = = = = = = = = = = = = = = = =
                
                ws.add_image(pic, get_column_letter(pic_index) + str(row))
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =
        
        if src != None and src != '':
            print(src +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
wb.save('./crawler-pic.xlsx')
print()
print('爬虫结束')

Src索引：11
Pic索引：8

https://www.rockauto.com/info/903/5S12715__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S15024__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S12716__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S16368__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S14989__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S16588__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S15030__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S15046__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S16591__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S12714__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S15035__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S15399__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S6989__ra_m.jpg  <->  [ok] - 剩余数量：0
https://www.rockauto.com/info/903/5S13515__ra_m

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
